In [40]:
# Define storage account and container
storage_account = "ds562team9datalake"
container_name = "silver"
file_path = "historical_crypto_tweet_data/historical_crypto_tweets_2014_2019.parquet"

storage_key = "KXg2Djg7uRevBSpPNIVnKw/N6HpqBh+kJwDX07wkywbpU2joMZdTIBOXk30EoMMxH2d8wwb+9j0g+AStO60IWw=="

spark.conf.set(f"fs.azure.account.key.{storage_account}.blob.core.windows.net", storage_key)
spark.conf.set("spark.hadoop.mapreduce.fileoutputcommitter.algorithm.version", "2")
spark.conf.set(
    "spark.sql.sources.commitProtocolClass",
    "org.apache.spark.internal.io.cloud.PathOutputCommitProtocol"
)
spark.conf.set(
    "spark.sql.parquet.output.committer.class",
    "org.apache.spark.sql.execution.datasources.parquet.DirectParquetOutputCommitter"
)

url = f"wasbs://{container_name}@{storage_account}.blob.core.windows.net/{file_path}"

In [41]:
df_raw = spark.read.parquet(url)

In [42]:
df_raw.printSchema()

In [43]:
df_raw.show(5, truncate=False)

In [44]:
df_cleaned = df_raw.drop("sentiment")

In [45]:
df_cleaned.show(5)

In [46]:
spark.conf.set(
    "fs.azure.account.key.ds562team9datalake.blob.core.windows.net",
    "KXg2Djg7uRevBSpPNIVnKw/N6HpqBh+kJwDX07wkywbpU2joMZdTIBOXk30EoMMxH2d8wwb+9j0g+AStO60IWw=="
)

In [48]:
# Fix the committer issue
spark.conf.set("spark.sql.parquet.output.committer.class", "org.apache.parquet.hadoop.ParquetOutputCommitter")

# Then proceed to write
df_cleaned.coalesce(1).write.mode("overwrite").parquet(
    "wasbs://bronze@ds562team9datalake.blob.core.windows.net/Historical_crypto_tweets/cleaned"
)